# Imports

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')
print(f'Keras version: {tf.keras.__version__}')

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

TensorFlow version: 2.3.1
Keras version: 2.4.0


In [13]:
print('GPU found :)' if tf.config.list_physical_devices("GPU") else 'No GPU :(')

No GPU :(


In [8]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout

# Dataset informations

5 classes: 
- Bowl --> train = 1019 images / test = 565 images
- CanOfCocaCola --> train = 874 images / test = 671 images
- MilkBottle --> train = 1024 images / test = 642 images
- Rice --> train = 1032 images / test = 877 images
- Sugar --> train = 787 images / test = 813

# Data Loading

In [12]:
preprocess = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input
)

train_generator = preprocess.flow_from_directory(
        'DB/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

test_generator = preprocess.flow_from_directory(
        'DB/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 4736 images belonging to 5 classes.
Found 3568 images belonging to 5 classes.


# Model creation and training

In [14]:
# Create our model from pretrained vgg19
vgg19 = VGG19(include_top=False, weights="imagenet")
vgg19.trainable = False

model = Sequential([
    Input((224, 224, 3)),
    vgg19,
    Flatten(),
    Dense(1024),
    Dropout(0.1),
    Dense(1024),
    Dense(5, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, None, None, 512)   20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 5125      
Total params: 46,770,245
Trainable params: 26,745,861
Non-trainable params: 20,024,384
___________________________________

In [ ]:
# Compile, train and save model

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=20, verbose=1, batch_size=32)

model.save("modele-VGG19.h5")

Epoch 1/20
148/148 [==============================] - 1958s 13s/step - loss: 14.4614 - accuracy: 0.9533
Epoch 2/20
148/148 [==============================] - 1876s 13s/step - loss: 5.5169 - accuracy: 0.9909
Epoch 3/20
148/148 [==============================] - 1860s 13s/step - loss: 12.3109 - accuracy: 0.9909
Epoch 4/20


In [ ]:
# Evaluate model performance on test data
_, test_acc = cnn_model.evaluate(test_generator, verbose=0)

print(f"Test accuracy: {test_acc:.5f}")